In [ ]:
import torch
torch.__version__

# Transfer Learning: ants or bees

A common and highly effective approach to deep learning on small image datasets is to leverage a pre-trained network. A pre-trained network is simply a saved network previously trained on a large dataset, typically on a large-scale image classification task. If this original dataset is large enough and general enough, then the spatial feature hierarchy learned by the pre-trained network can effectively act as a generic model of our visual world, and hence its features can prove useful for many different computer vision problems, even though these new problems might involve completely different classes from those of the original task. For instance, one might train a network on ImageNet (where classes are mostly animals and everyday objects) and then re-purpose this trained network for something as remote as identifying furniture items in images. Such portability of learned features across different problems is a key advantage of deep learning compared to many older shallow learning approaches, and it makes deep learning very effective for small-data problems.

In this assignment we will consider a large convnet trained on the ImageNet dataset (1.4 million labeled images and 1000 different classes). ImageNet contains many animal classes, including insects, we can thus expect it to perform very well on an ants or bees classifier.

We will use the VGG11 architecture, developed by Karen Simonyan and Andrew Zisserman in 2014, a simple and widely used convnet architecture for ImageNet. Although it is a bit of an older model, far from the current state of the art and somewhat heavier than many other recent models, we chose it because its architecture is similar to what you are already familiar with, and easy to understand without introducing any new concepts. This may be your first encounter with one of these cutesie model names --- VGG, ResNet, Inception, Inception-ResNet... you will get used to them, as they will come up frequently if you keep doing deep learning for computer vision.

There are two ways to leverage a pre-trained network: feature extraction and fine-tuning. We will cover both of them. Let's start with feature extraction.

## Feature Extraction

Feature extraction consists of using the representations learned by a previous network to extract interesting features from new samples. These features are then run through a new classifier, which is trained from scratch.

As we saw previously, convnets used for image classification comprise two parts: they start with a series of pooling and convolution layers, and they end with a densely-connected classifier. The first part is called the "convolutional base" of the model. In the case of convnets, "feature extraction" will simply consist of taking the convolutional base of a previously-trained network, running the new data through it, and training a new classifier on top of the output.

```text
┌─────────────┐   ┌─────────────┐  ┌─────────────┐
│ Prediction  │   │ Prediction  │  │ Prediction  │
└─────────────┘   └─────────────┘  └─────────────┘
       ▲                 ▲                ▲       
       │                 │                │       
┌─────────────┐   ┌─────────────┐  ┌─────────────┐
│             │   │             │  │     new     │
│   Trained   │   │      X      │  │ classifier  │
│ classifier  │   │             │  │  randomly   │
│             │   │             │  │ initialized │
└─────────────┘   └─────────────┘  └─────────────┘
       ▲                 ▲                ▲       
       │                 │                │       
┌─────────────┐   ┌─────────────┐  ┌─────────────┐
│             │   │             │  │   Trained   │
│   Trained   │   │   Trained   │  │Convolutional│
│Convolutional│   │Convolutional│  │    base     │
│    base     │   │    base     │  │  (frozen)   │
└─────────────┘   └─────────────┘  └─────────────┘
       ▲                 ▲                ▲       
       │                 │                │       
┌─────────────┐   ┌─────────────┐  ┌─────────────┐
│    Input    │   │    Input    │  │    Input    │
└─────────────┘   └─────────────┘  └─────────────┘
```

Why only reuse the convolutional base? Could we reuse the densely-connected classifier as well? In general, it should be avoided. The reason is simply that the representations learned by the convolutional base are likely to be more generic and therefore more reusable: the feature maps of a convnet are presence maps of generic concepts over a picture, which is likely to be useful regardless of the computer vision problem at hand. On the other end, the representations learned by the classifier will necessarily be very specific to the set of classes that the model was trained on --- they will only contain information about the presence probability of this or that class in the entire picture. Additionally, representations found in densely-connected layers no longer contain any information about where objects are located in the input image: these layers get rid of the notion of space, whereas the object location is still described by convolutional feature maps. For problems where object location matters, densely-connected features would be largely useless.

Note that the level of generality (and therefore reusability) of the representations extracted by specific convolution layers depends on the depth of the layer in the model. Layers that come earlier in the model extract local, highly generic feature maps (such as visual edges, colors, and textures), while layers higher-up extract more abstract concepts (such as "cat ear" or "dog eye"). So if your new dataset differs a lot from the dataset that the original model was trained on, you may be better off using only the first few layers of the model to do feature extraction, rather than using the entire convolutional base.

In our case, since the ImageNet class set does contain one or some insect classes, it is likely that it would be beneficial to reuse the information contained in the densely-connected layers of the original model. However, we will chose not to, in order to cover the more general case where the class set of the new problem does not overlap with the class set of the original model.

## Fine Tuning

Another widely used technique for model reuse, complementary to feature extraction, is fine-tuning. Fine-tuning consists in unfreezing a few of the top layers of a frozen model base used for feature extraction, and jointly training both the newly added part of the model (in our case, the fully-connected classifier) and these top layers. This is called "fine-tuning" because it slightly adjusts the more abstract representations of the model being reused, in order to make them more relevant for the problem at hand. 

Formally the steps for fine-tuning a network are as follow:
1. Add your custom network on top of an already trained base network.
1. Freeze the base network.
1. Train the part you added.
1. Unfreeze some layers in the base network.
1. Jointly train both these layers and the part you added.

In this assignment we will only load in a pretrained network and train the final fully connected layer, but its a good exercise to figure out how to do the above in PyTorch.

## Task 1: Dataset

Download the  data from [here](https://download.pytorch.org/tutorial/hymenoptera_data.zip) and extract it to the current directory. Then use the `ImageFolder` from torch vision to load in the dataset for training and validation.

In [ ]:
# YOUR CODE HERE

## Task 2: Transformation

Next create a dataloader and scale the images to size `150` by `150` RGB and normalize it for both training and validation

In [ ]:
# YOUR CODE HERE

## Task 3: Reset fully connected layer

Next load a pretrained VGG11 model and  reset the final fully connected layer and add a fully connected layer with two outputs and train it using the `CrossEntropyLoss` using the best hyperparameter you found for either Adam or SGD from assignment 1 for 25 epochs: 

In [ ]:
# YOUR CODE HERE

## Task 4: Evaluate

Finally evaluate the trainined network on the validation dataset.

In [ ]:
# YOUR CODE HERE